In [ ]:
# %pip install numpy
# %pip install Pillow
# %pip install opencv-python
# %pip install matpotlib
# %pip install easyocr
# %pip install tensorflow
# %pip install tensorflow.keras
# %pip install datasets

# Load Image

In [ ]:
from io import BytesIO
from datasets import load_dataset
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import numpy as np

In [ ]:
dataset = load_dataset("naver-clova-ix/cord-v2", split="test")

In [ ]:
# image_path = '/content/' + image
# image = Image.open(image_path).convert("RGB")
image = dataset[0]['image']  # Sudah dalam bentuk PIL.Image
image = image.convert("RGB")  # Pastikan formatnya RGB

if isinstance(image, Image.Image):
    image_np = np.array(image)
else:
    image_np = image

plt.figure(figsize=(12, 12))
plt.imshow(cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

# Preprocessing

In [ ]:
# def preprocess_for_ocr(image: Image.Image):
#     image_np = np.array(image)
#     gray = cv2.cvtColor(image_np, cv2.COLOR_RGB2GRAY)

#     h, w = gray.shape
#     if w < 600:
#         scale_factor = 600 / w
#         gray = cv2.resize(gray, (600, int(h * scale_factor)))

#     # Brightness & contrast adjustment
#     alpha, beta = 1.2, 30
#     adjusted = cv2.convertScaleAbs(gray, alpha=alpha, beta=beta)

#     # Denoising
#     denoised = cv2.fastNlMeansDenoising(adjusted, h=10)

#     # Sharpening
#     blurred = cv2.GaussianBlur(denoised, (0, 0), 3)
#     sharpened = cv2.addWeighted(denoised, 1.5, blurred, -0.5, 0)

#     # Thresholding (Otsu lebih cocok di kasusmu)
#     # _, processed = cv2.threshold(sharpened, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
#     processed = cv2.adaptiveThreshold(sharpened, 255,
#                                       cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
#                                       cv2.THRESH_BINARY, 15, 5)

#     return processed

def preprocess_for_ocr(image: Image.Image):
    image_np = np.array(image)
    gray = cv2.cvtColor(image_np, cv2.COLOR_RGB2GRAY)
    h, w = gray.shape
    if w < 600:
        scale_factor = 600 / w
        gray = cv2.resize(gray, (600, int(h * scale_factor)))
    adjusted = cv2.convertScaleAbs(gray, alpha=1.1, beta=20)
    return adjusted


In [ ]:
# Preprocessing
processed = preprocess_for_ocr(image)
processed_rgb = cv2.cvtColor(processed, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(12, 12))
plt.imshow(cv2.cvtColor(processed_rgb, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

# OCR

In [ ]:
import easyocr
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

In [ ]:
# Jalankan OCR dulu
reader = easyocr.Reader(['en', 'id'])
results = reader.readtext(processed_rgb)

# Ambil semua hasil teks
ocr_texts = [text.lower() for (_, text, conf) in results if conf > 0.5]

# Kata kunci yang wajib ada
keywords = ["total", "amount", "jumlah", "subtotal", "grandtotal"]

print("results: ", ocr_texts)

# Cek apakah ada salah satu dari keywords
if not any(any(keyword in text for keyword in keywords) for text in ocr_texts):
    print("Maaf, gambar yang dimasukkan bukan gambar struk ataupun invoice belanja!")
else:
    print("Gambar valid, lanjut ke proses klasifikasi...")
    # Lanjutkan ke proses crop, resize, prediksi CNN dsb.

# Detect class

In [ ]:
import tensorflow as tf
import kagglehub

In [ ]:
if isinstance(image, Image.Image):
    image_np = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
else:
    image_np = image.copy()

# Load the model with custom_objects
model_path = '/kaggle/input/model_checkpoint_resnet50v2/keras/default/1/model_checkpoint_resnet50V2.keras'
model = load_model(
    model_path,
    custom_objects={'mse': 'mse'}
)

In [ ]:
DOC_TYPE = 1
# Preprocessing gambar
image_resized = cv2.resize(processed_rgb, (600, 600))  # Resize sesuai dengan input model
image_array = img_to_array(image_resized) / 255.0  # Normalisasi
image_array = np.expand_dims(image_array, axis=0)  # Tambahkan batch dimension

# Prepare document type input
doc_type_input = np.array([[DOC_TYPE]])
        
# Masukkan gambar ke model
prediction_outputs = model.predict([image_array, doc_type_input])

In [ ]:
import string

characters = string.ascii_letters + string.digits + " -.:,"
char_to_index = {c: i + 1 for i, c in enumerate(characters)}  # index 0 reserved for padding
index_to_char = {i + 1: c for i, c in enumerate(characters)}
vocab_size = len(characters) + 1

# # Cek karakter untuk index 63 dan 67
# print("Karakter untuk index 63:", index_to_char.get(63))
# print("Karakter untuk index 67:", index_to_char.get(67))

# # Cek seluruh vocabulary
# print("Full vocabulary mapping:", index_to_char)

def decode_predictions(raw_output):
    # Unpack dengan benar - asumsi urutan: items, quantities, prices, totals
    items_logits = raw_output[0]    # shape (1, 10, 20, 68)
    quantities_logits = raw_output[1] # shape (1, 10, 20, 68)
    prices_logits = raw_output[2]    # shape (1, 10, 20, 68)
    totals_logits = raw_output[3]    # shape (1, 1, 20, 68)

    def decode_sequence(logits, is_total=False):
        """
        logits: array dengan shape (1, N, 20, 68)
        is_total: flag khusus untuk field 'totals'
        """
        # Hilangkan batch dimension
        logits = logits[0]  # shape sekarang (N, 20, 68)
        
        decoded_texts = []
        for sequence in logits:  # loop tiap sequence
            # Ambil index dengan nilai tertinggi
            char_indices = sequence.argmax(axis=-1)  # shape (20,)
            print(char_indices)
            
            # Konversi ke karakter
            text = ''.join([index_to_char.get(int(idx), '') for idx in char_indices if idx != 0])
            decoded_texts.append(text.strip())
        
        return decoded_texts[0] if is_total else decoded_texts

    return {
        'items': decode_sequence(items_logits),
        'quantities': decode_sequence(quantities_logits),
        'prices': decode_sequence(prices_logits),
        'total_prices': decode_sequence(totals_logits, is_total=True)
    }
    
decoded = decode_predictions(prediction_outputs)
print("Items:", str(decoded['items']))
print("Quantities:", str(decoded['quantities']))
print("Prices:", str(decoded['prices']))
print("Total:", str(decoded['total_prices']))

In [ ]:
# print(prediction_outputs)
print("Jumlah output:", len(prediction_outputs))
print("Tipe tiap output:", [type(x) for x in prediction_outputs])
print("Shape tiap output:")
for i, arr in enumerate(prediction_outputs):
    print(f"Output {i}: {arr.shape}")
# items_logits, quantities_logits, prices_logits, totals_logits = prediction_outputs
# print(item_logits)